In [1]:
import cv2
import mediapipe as mp
import urllib.request
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import PyQt5
from PIL import Image
from IPython.display import Video
import nb_helpers
import time
import random as rand
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
#mp_face_mesh = mp.solutions.face_mesh

In [3]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

In [4]:
from win32api import GetSystemMetrics
w =  GetSystemMetrics(0)
h = GetSystemMetrics(1)
print("Width =", GetSystemMetrics(0))
print("Height =", GetSystemMetrics(1))

Width = 1280
Height = 720


In [5]:
def inImage(image,points):
    for point in points:
        x,y = point
        im = Image.open(image) # Can be many different formats.
        pix = im.load()
        colour = pix[x,y]
        #print(colour)
        if colour != (0,0,0):
            return False
    
    return True

#print(inImage("pgreen.jpg",[(150,150),(0,0)]))

In [6]:
path = "C:\\Computer\\Programming\\opencv\\c\\poses"
pos = os.listdir(path)
poses = []
for p in pos:
    if p.endswith("jpg"):
        poses.append(p)
print(poses)

['0.jpg', '1.jpg']


In [7]:
def nextname():
    return str(int(max(poses)[:-4])+1)
print(nextname())

2


In [8]:
def drawline(layer,a,b,colour,width):
    cv2.line(layer, (int(a.x*w), int(a.y*h)), (int(b.x*w), int(b.y*h)), colour, width)

In [9]:
def centercoor(text,scale,thicknes):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = scale
    thickness = thicknes
    text_width, text_height = cv2.getTextSize(text, fontFace, fontScale, thickness)[0]
    return (int(w / 2) - int(text_width / 2), int(h / 2) + int(text_height / 2))

In [10]:
import cv2
import numpy as np
import mediapipe
from keras.models import load_model

# Load the pre-trained model
model = load_model('my_model3.h5')

# Define a dictionary to map the model's output to the corresponding emotion
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}

In [11]:
cap = cv2.VideoCapture(0)

currimage = ""
start = True
end = False
menu = False
turret = False
timed = False
e = False
lives = 0
score =0
makepose = False 
startTime = float("INF")
while True:
    #print("Running")

    if cv2.waitKey(1)==ord('q') or e:
        break

    success, img = cap.read()

    img = cv2.resize(img, (w,h))
    img = cv2.flip(img, 1)
    imgPlayer = img

    landDetected = True

    

    #imgPlayer = np.zeros((512,512,3), np.uint8)
    #imgPlayer = cv2.resize(imgPlayer, (w,h))
   

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    image = cv2.imread(currimage)

    try: 
        lm = results.pose_landmarks.landmark[11:17] + results.pose_landmarks.landmark[19:21]
    except: 
        landDetected = False

    if landDetected: 
        error = int(w / 13)
        #print("landDetected")


        lm = results.pose_landmarks.landmark
        lShould = lm[12]
        lElb = lm[14]
        lWrist = lm[16]
        lInd = lm[20]
        lWaist = lm[24]

        rShould = lm[11]
        rElb = lm[13]
        rWrist = lm[15]
        rInd = lm[19]
        rWaist = lm[23]

        head = lm[0]

        limbs = [lShould,lElb,lWrist,lInd,rShould,rElb,rWrist,rInd,rWaist,lWaist,head]
        points = [(l.x,l.y) for l in limbs]
        
        blue = (255,0,0)
        linewidth = 5
        #cv2.line(imgPlayer, (int(lShould.x*w), int(lShould.y*h)), (int(lElb.x*w), int(lElb.y*h)), (255,0,0), 5)
        drawline(imgPlayer,lShould,lElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(lElb.x*w), int(lElb.y*h)), (int(lWrist.x*w), int(lWrist.y*h)), (255,0,0), 5)
        drawline(imgPlayer,lWrist,lElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(lWrist.x*w), int(lWrist.y*h)), (int(lInd.x*w), int(lInd.y*h)), (255,0,0), 5)
        drawline(imgPlayer,lWrist,lInd,blue,linewidth)

        #cv2.line(imgPlayer, (int(rShould.x*w), int(rShould.y*h)), (int(rElb.x*w), int(rElb.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rShould,rElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(rElb.x*w), int(rElb.y*h)), (int(rWrist.x*w), int(rWrist.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rWrist,rElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(rWrist.x*w), int(rWrist.y*h)), (int(rInd.x*w), int(rInd.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rWrist,rInd,blue,linewidth)

        #cv2.line(imgPlayer, (int(lWaist.x*w), int(lWaist.y*h)), (int(rWaist.x*w), int(rWaist.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rWaist,lWaist,blue,linewidth)
        drawline(imgPlayer,rShould,lShould,blue,linewidth)
        drawline(imgPlayer,rShould,rWaist,blue,linewidth)
        drawline(imgPlayer,lShould,lWaist,blue,linewidth)

        #drawline(imgPlayer,head,lShould,blue,linewidth)
        #drawline(imgPlayer,head,rShould,blue,linewidth)
        for point in points:
            x,y = point
            if x <= 0 or x >= 1 or y <= 0 or y >=1:
                continue
            x = int(x*w)
            y = int(y*h)

            cv2.circle(imgPlayer, (x, y), 5, (255,0,0), cv2.FILLED)
        

        if start:
            currimage = "C:\\Computer\\Programming\\opencv\\c\\poses\\default\\start.jpg"
            cv2.putText(imgPlayer, "Line up to start", (centercoor("Line up to start",3,3)[0],75), cv2.FONT_HERSHEY_SIMPLEX,3,(255,255,255), 3)
            

            # Create the condition to check if each pixel in the original image is black
            overlay = cv2.resize(cv2.imread(currimage),(w,h))
            overlay = cv2.flip(overlay,1)

            imgPlayer = cv2.addWeighted(imgPlayer,0.4,overlay,0.1,0)


            isin = True
            for point in points:
                x,y = point
                if x <= 0 or x >= 1 or y <= 0 or y >=1:
                    continue
                x = int(x*w)
                y = int(y*h)
                im = cv2.imread(currimage) # Can be many different formats.
                im = cv2.flip(im, 1)

                if (im[y,x] == [0]).all():
                    cv2.circle(imgPlayer, (x, y), 5, (0,255,0), cv2.FILLED)
                else:
                    cv2.circle(imgPlayer, (x, y), 5, (0,0,255), cv2.FILLED)
                    isin = False

            if isin:
                start = False
                menu = True
            
        elif menu:
            try:
                rectangles = {"Timed":((100,100), (200, 200)),
                              "Exit":((1080,100), (1180, 200)), 
                              "Turret":((w//2,100), (w//2+100, 200)),
                              "Make your own pose!":((100,300), (200, 400))}
                for key,val in rectangles.items():
                    cv2.rectangle(imgPlayer, val[0], val[1], (255, 0, 0), 2)
                    cv2.putText(imgPlayer, key, (val[0][0],(val[0][1]+val[1][1])//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    if val[0][0]<=int(head.x*w)<=val[1][0] and val[0][1]<=int(head.y*h)<=val[1][1]:
                        menu = False
                        if key == "Timed":
                            timed = True
                            startTime = time.time()
                            score = 0
                        elif key == "Exit":
                            e = True
                        elif key == "Turret":
                            turret = True
                            lives = 3
                            score = 0
                            startTime = time.time()
                        
                        elif key == "Make your own pose!":
                            makepose = True
                            startTime = time.time()
                
            except:
                pass
        
        elif makepose:
            curTime = time.time()
            timeleft = curTime-startTime
            cv2.putText(imgPlayer, "Time Remaining: " + str(3-timeleft) + "s", (800, 50), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 3)
            if timeleft > 3:
                frame = img.copy()
                with mp_pose.Pose(static_image_mode=True, 
                    model_complexity=2,
                    enable_segmentation=True) as pp:
                    
                    # Read the file in and get dims
                    #image = cv2.imread(frame)
                    #display(image)

                    # Convert the BGR image to RGB and then process with the `Pose` object.
                    results = pp.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                seg = frame.copy()
                tightness = 0.3

                # Stack the segmentation mask for 3 RGB channels, and then create a filter for which pixels to keep

                if np.stack((results.segmentation_mask,) * 3, axis=-1).any() == None: 
                    continue

                condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > tightness

                # Creates a black background image
                bg_image = np.zeros(frame.shape, dtype=np.uint8)
                figure = np.zeros(frame.shape, dtype=np.uint8)

                # Can change the color of this background by specifying (0-255) RGB values. We choose green-screen green.
                bg_image[:] = [255, 255, 255]
                figure[:] = [0, 0, 0]

                # For every pixel location, display the corresponding pixel from the original imgae if the condition in our filter is met (i.e. the probability of being part of the object is above the inclusiogn threshold), or else display corresponding pixel from the background array (i.e. green)
                seg = np.where(condition, figure, bg_image)
                seg = cv2.resize(seg, (w,h))

                filename = nextname()+".jpg"

                cv2.imwrite(path+"\\"+filename, seg)
                
                makepose = False
                menu = True

        elif turret:
            if lives <= 0:
                turret = False
                end = True
            curTime = time.time()
            timeleft = round(3 - (curTime - startTime), 2)
            cv2.putText(imgPlayer, "Score: "+str(score), (50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255), 4)
            cv2.putText(imgPlayer, "Lives Remaining: " + str(lives), (800, 50), cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,255), 3)
            cv2.putText(imgPlayer,  str(timeleft), centercoor(str(timeleft),3,3), cv2.FONT_HERSHEY_SIMPLEX,3, (255,255,255), 3)

            overlay = cv2.resize(cv2.imread(currimage),(w,h))
            overlay = cv2.flip(overlay,1)

            imgPlayer = cv2.addWeighted(imgPlayer,0.4,overlay,0.1,0)
            if timeleft <=0:
                isin = True
                for point in points:
                    x,y = point
                    if x <= 0 or x >= 1 or y <= 0 or y >=1:
                        continue
                    x = int(x*w)
                    y = int(y*h)
                    im = cv2.imread(currimage) # Can be many different formats.
                    im = cv2.flip(im, 1)

                    if (im[y,x] == [0]).all():
                        cv2.circle(imgPlayer, (x, y), 5, (0,255,0), cv2.FILLED)
                    else:
                        cv2.circle(imgPlayer, (x, y), 5, (0,0,255), cv2.FILLED)
                        isin = False

                if isin:
                    score += 1
                else:
                    lives -= 1
                currimage = "C:\\Computer\\Programming\\opencv\\c\\poses\\"+str(rand.choice(poses))
                time.sleep(1)

                startTime = time.time()

        elif end:
            cv2.putText(imgPlayer, "Score: "+str(score), centercoor("Score: "+str(score),4,4), cv2.FONT_HERSHEY_SIMPLEX,4,(255,255,255), 4)
            
            try:
                rectangles = {"Menu":((100,520), (200, 620))}
                for key,val in rectangles.items():
                    cv2.rectangle(imgPlayer, val[0], val[1], (255, 0, 0), 2)
                    cv2.putText(imgPlayer, key, (val[0][0],(val[0][1]+val[1][1])//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    if val[0][0]<=int(head.x*w)<=val[1][0] and val[0][1]<=int(head.y*h)<=val[1][1]:
                        end = False
                        if key == "Menu":
                            menu = True
                            timed = False
                
            except:
                pass
            
            cv2.imshow("Image Player", imgPlayer)
            if cv2.waitKey(1)==ord('q'):
                break
            #print(end)
        elif timed:
            curTime = time.time()
            timeleft = round(30 - (curTime - startTime), 2)
            if timeleft <= 0:
                timed = False
                end = True

            cv2.putText(imgPlayer, "Score: "+str(score), (50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255), 4)
            cv2.putText(imgPlayer, "Time Remaining: " + str(timeleft) + "s", (800, 50), cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,255), 3)


            # Create the condition to check if each pixel in the original image is black
            #print(currimage)
            overlay = cv2.resize(cv2.imread(currimage),(w,h))
            overlay = cv2.flip(overlay,1)

            imgPlayer = cv2.addWeighted(imgPlayer,0.4,overlay,0.1,0)

            isin = True
            for point in points:
                x,y = point
                if x <= 0 or x >= 1 or y <= 0 or y >=1:
                    continue
                x = int(x*w)
                y = int(y*h)
                im = cv2.imread(currimage) # Can be many different formats.
                im = cv2.flip(im, 1)

                if (im[y,x] == [0]).all():
                    cv2.circle(imgPlayer, (x, y), 5, (0,255,0), cv2.FILLED)
                else:
                    cv2.circle(imgPlayer, (x, y), 5, (0,0,255), cv2.FILLED)
                    isin = False

            if isin:
                score += 1
                currimage = "C:\\Computer\\Programming\\opencv\\c\\poses\\"+str(rand.choice(poses))
                time.sleep(0.5)

            

    cv2.imshow("Image Player", imgPlayer)


cv2.destroyAllWindows()
cap = None

c:\Users\Oat_M\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
